# Description

Reads the gene pair samples across different categories and computes their p-values.

# Modules loading

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import pandas as pd
from concurrent.futures import as_completed, ProcessPoolExecutor
from tqdm import tqdm
from pathlib import Path

from ccc.coef import ccc
from ccc import conf

# Settings

In [2]:
DATASET_CONFIG = conf.GTEX
GTEX_TISSUE = "whole_blood"
GENE_SEL_STRATEGY = "var_pc_log2"

PVALUE_N_PERMS = 1000

RANDOM_STATE = np.random.RandomState(0)

# Configuration constants
TOP_N_GENES = "all"
DATA_DIR = Path("/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8")
GENE_SELECTION_DIR = DATA_DIR / "gene_selection" / TOP_N_GENES
SIMILARITY_MATRICES_DIR = DATA_DIR / "similarity_matrices" / TOP_N_GENES

# Paths

In [3]:
INPUT_GENE_EXPR_FILE = (
    GENE_SELECTION_DIR / f"gtex_v8_data_{GTEX_TISSUE}-{GENE_SEL_STRATEGY}.pkl"
)
display(INPUT_GENE_EXPR_FILE)

assert INPUT_GENE_EXPR_FILE.exists()

PosixPath('/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8/gene_selection/all/gtex_v8_data_whole_blood-var_pc_log2.pkl')

In [4]:
INPUT_GENE_PAIRS_INTERSECTIONS_FILE = Path(
    "/mnt/data/projs/manuscripts/ccc-gpu/results/gene_pair_intersections-gtex_v8-whole_blood-var_pc_log2.pkl"
)
display(INPUT_GENE_PAIRS_INTERSECTIONS_FILE)

assert INPUT_GENE_PAIRS_INTERSECTIONS_FILE.exists()

PosixPath('/mnt/data/projs/manuscripts/ccc-gpu/results/gene_pair_intersections-gtex_v8-whole_blood-var_pc_log2.pkl')

In [5]:
OUTPUT_DIR = Path("/mnt/data/projs/manuscripts/ccc-gpu/results/") / "pvalues"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [6]:
OUTPUT_DIR

PosixPath('/mnt/data/projs/manuscripts/ccc-gpu/results/pvalues')

# Load gene expression data

In [7]:
data = pd.read_pickle(INPUT_GENE_EXPR_FILE).sort_index()

In [8]:
data.shape

(56200, 755)

# Load gene pairs samples

In [9]:
output_file = OUTPUT_DIR / "gene_pair-samples.pkl"

In [10]:
gene_pair_samples = pd.read_pickle(output_file)

In [11]:
len(gene_pair_samples)

34

In [12]:
sorted(gene_pair_samples.keys())

['all_high-random',
 'all_high-top_ccc',
 'all_high-top_pearson',
 'all_high-top_spearman',
 'all_low-random',
 'all_low-top_ccc',
 'all_low-top_pearson',
 'all_low-top_spearman',
 'ccc_high_and_pearson_low-random',
 'ccc_high_and_pearson_low-top_ccc',
 'ccc_high_and_pearson_low-top_pearson',
 'ccc_high_and_pearson_low-top_spearman',
 'ccc_high_and_spearman_low-random',
 'ccc_high_and_spearman_low-top_ccc',
 'ccc_high_and_spearman_low-top_pearson',
 'ccc_high_and_spearman_low-top_spearman',
 'ccc_high_and_spearman_pearson_low-random',
 'ccc_high_and_spearman_pearson_low-top_ccc',
 'ccc_high_and_spearman_pearson_low-top_pearson',
 'ccc_high_and_spearman_pearson_low-top_spearman',
 'ccc_spearman_high_and_pearson_low-random',
 'ccc_spearman_high_and_pearson_low-top_ccc',
 'ccc_spearman_high_and_pearson_low-top_pearson',
 'ccc_spearman_high_and_pearson_low-top_spearman',
 'entire_dataset-random',
 'pearson_high_and_ccc_low-random',
 'pearson_high_and_ccc_low-top_ccc',
 'pearson_high_and_cc

In [13]:
_k = list(gene_pair_samples.keys())[0]
gene_pair_samples[_k].head()

Pearson (high)  Pearson (low)  \
ENSG00000255945.1 ENSG00000232604.1            True          False   
                  ENSG00000257296.1            True          False   
ENSG00000256281.1 ENSG00000267687.1            True          False   
ENSG00000284356.1 ENSG00000248928.1            True          False   
ENSG00000283680.1 ENSG00000278497.1            True          False   

                                     Spearman (high)  Spearman (low)  \
ENSG00000255945.1 ENSG00000232604.1             True           False   
                  ENSG00000257296.1             True           False   
ENSG00000256281.1 ENSG00000267687.1             True           False   
ENSG00000284356.1 ENSG00000248928.1             True           False   
ENSG00000283680.1 ENSG00000278497.1             True           False   

                                     CCC (high)  CCC (low)  ccc  pearson  \
ENSG00000255945.1 ENSG00000232604.1        True      False  1.0      1.0   
                  ENSG00000257296.1        True      False  1.0      1.0   
ENSG00000256281.1 ENSG00000267687.1        True      False  1.0      1.0   
ENSG00000284356.1 ENSG00000248928.1        True      False  1.0      1.0   
ENSG00000283680.1 ENSG00000278497.1        True      False  1.0      1.0   

                                     spearman  
ENSG00000255945.1 ENSG00000232604.1       1.0  
                  ENSG00000257296.1       1.0  
ENSG00000256281.1 ENSG00000267687.1       1.0  
ENSG00000284356.1 ENSG00000248928.1       1.0  
ENSG00000283680.1 ENSG00000278497.1       1.0

In [14]:
[i for i in gene_pair_samples[_k].head(10).index]

[('ENSG00000255945.1', 'ENSG00000232604.1'),
 ('ENSG00000255945.1', 'ENSG00000257296.1'),
 ('ENSG00000256281.1', 'ENSG00000267687.1'),
 ('ENSG00000284356.1', 'ENSG00000248928.1'),
 ('ENSG00000283680.1', 'ENSG00000278497.1'),
 ('ENSG00000255555.1', 'ENSG00000230840.1'),
 ('ENSG00000278988.1', 'ENSG00000219666.2'),
 ('ENSG00000202160.1', 'ENSG00000252554.1'),
 ('ENSG00000202160.1', 'ENSG00000263967.1'),
 ('ENSG00000280580.1', 'ENSG00000278381.1')]

# Compute pvalues on sampled gene pairs

In [15]:
output_file = OUTPUT_DIR / "gene_pair-samples-pvalues.pkl"

In [16]:
def corr_single(x, y):
    ccc_val, ccc_pval = ccc(x, y, pvalue_n_perms=PVALUE_N_PERMS, n_jobs=12)
    p_val, p_pval = stats.pearsonr(x, y)
    s_val, s_pval = stats.spearmanr(x, y)

    return ccc_val, ccc_pval, p_val, p_pval, s_val, s_pval

In [ ]:
results = []

# I leave the ProcessPoolExecutor here in case I want to easily swith between
# parallelize across gene pairs (max_workers=conf.GENERAL["N_JOBS"] and n_jobs=1 inside function corr_single)
# or across permutations for one gene pair (max_workers=1 and n_jobs=conf.GENERAL["N_JOBS"])
with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(corr_single, data.loc[gene0], data.loc[gene1]): (
            gene0,
            gene1,
            k,
        )
        for k, v in gene_pair_samples.items()
        for gene0, gene1 in gene_pair_samples[k].index
    }

    for t_idx, t in tqdm(enumerate(as_completed(tasks)), total=len(tasks), ncols=100):
        gene0, gene1, k = tasks[t]
        ccc_val, ccc_pval, p_val, p_pval, s_val, s_pval = t.result()

        results.append(
            {
                "gene0": gene0,
                "gene1": gene1,
                "group": k,
                "ccc": ccc_val,
                "ccc_pvalue": ccc_pval,
                "pearson": p_val,
                "pearson_pvalue": p_pval,
                "spearman": s_val,
                "spearman_pvalue": s_pval,
            }
        )

        # save
        _df = pd.DataFrame(results)
        _df["group"] = _df["group"].astype("category")
        _df.to_pickle(output_file)

In [18]:
len(results)

17008

In [19]:
results_df = pd.DataFrame(results)
results_df["group"] = results_df["group"].astype("category")

In [20]:
results_df.shape

(17008, 9)

In [21]:
results_df.head()

,gene0,gene1,group,ccc,ccc_pvalue,pearson,pearson_pvalue,spearman,spearman_pvalue
0,ENSG00000255555.1,ENSG00000230840.1,all_high-top_ccc,1.0,0.001998,1.0,0.0,1.0,0.0
1,ENSG00000255945.1,ENSG00000257296.1,all_high-top_ccc,1.0,0.003996,1.0,0.0,1.0,0.0
2,ENSG00000284356.1,ENSG00000248928.1,all_high-top_ccc,1.0,0.003996,1.0,0.0,1.0,0.0
3,ENSG00000256281.1,ENSG00000267687.1,all_high-top_ccc,1.0,0.000999,1.0,0.0,1.0,0.0
4,ENSG00000255945.1,ENSG00000232604.1,all_high-top_ccc,1.0,0.002997,1.0,0.0,1.0,0.0


# Remove duplicated gene pairs

This could happen when gene pairs overlap and are the top of different coefficients.

In [22]:
# results_df = results_df.drop_duplicates(subset=["gene0", "gene1"])

In [23]:
# results.shape

# Save

In [24]:
results_df.to_pickle(output_file)